# Transparency Dashboard Analysis

This notebook analyzes budget execution and programmatic indicators from Carmen de Areco's transparency portal.

In [ ]:
# Cell 1: Setup
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Ensure output dir
os.makedirs("../public/charts", exist_ok=True)

In [ ]:
# Cell 2: Load Data
df_2019 = pd.read_csv("../data/processed/budget_execution_2019.csv")
df_2021 = pd.read_csv("../data/processed/budget_execution_2021.csv")
df_ind = pd.read_csv("../data/processed/programmatic_indicators.csv")

In [ ]:
# Cell 3: Chart 1 – 2019 Budget vs Execution by Sector
fig1 = px.bar(
    df_2019,
    x="sector",
    y=["budget", "execution"],
    title="Ejecución Presupuestaria 2019 por Sector",
    labels={"value": "Monto (ARS)", "variable": "Tipo"},
    barmode="group",
    color_discrete_sequence=["#1f77b4", "#ff7f0e"]
)
fig1.update_layout(xaxis_tickangle=-45)
fig1.show()
fig1.write_html("../public/charts/budget_2019.html")

In [ ]:
# Cell 4: Chart 2 – 2021 Quarterly Execution Trend
df_q = df_2021[df_2021["Quarter"] != "Total"]
fig2 = go.Figure()
fig2.add_trace(go.Bar(x=df_q["Quarter"], y=df_q["Budgeted"], name="Presupuestado"))
fig2.add_trace(go.Bar(x=df_q["Quarter"], y=df_q["Executed"], name="Ejecutado"))
fig2.add_trace(go.Scatter(x=df_q["Quarter"], y=df_q["Percentage"], 
                         mode="lines+markers", name="% Ejecución", yaxis="y2"))

fig2.update_layout(
    title="Ejecución Trimestral 2021",
    yaxis=dict(title="Monto (ARS)"),
    yaxis2=dict(title="% Ejecución", overlaying="y", side="right"),
    barmode="group"
)
fig2.show()
fig2.write_html("../public/charts/budget_2021_quarterly.html")

In [ ]:
# Cell 5: Chart 3 – Security Cameras Over Time
df_cam = df_ind[df_ind["indicator"] == "security_cameras"].dropna(subset=["executed"])
df_cam = df_cam.sort_values("source_file")

fig3 = px.line(
    df_cam,
    x="source_file",
    y=["planned", "executed"],
    title="Cámaras de Seguridad: Plan vs Ejecución",
    markers=True,
    labels={"value": "Cantidad", "variable": "Tipo"}
)
fig3.update_layout(xaxis_title="Reporte (PDF)", xaxis_tickangle=-45)
fig3.show()
fig3.write_html("../public/charts/cameras_timeline.html")

# Insight: In Q4 2023, 298 cameras were installed vs 198 planned → 150% execution

In [ ]:
# Cell 6: Chart 4 – Families Assisted
df_fam = df_ind[df_ind["indicator"] == "families_assisted"].dropna(subset=["executed"])
df_fam = df_fam.sort_values("source_file")

fig4 = px.bar(
    df_fam,
    x="source_file",
    y="executed",
    title="Familias Asistidas por Programa",
    labels={"executed": "Cantidad", "source_file": "Reporte"}
)
fig4.update_layout(xaxis_tickangle=-45)
fig4.show()
fig4.write_html("../public/charts/families_assisted.html")

In [ ]:
# Cell 7: (Optional) Gender Staffing – If you parse it later
# You can add this once you extract gender data from PDF headers like 
# “PLANTEL DEL PERSONAL DESAGREGADO SEGÚN GÉNERO”